In [1]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt

%matplotlib inline

## EDA

### 1. Оценить количество фильмов, пользователей и оценок:

In [2]:
users = pd.read_csv('ml-1m/users.dat', sep='::', names=['user', 'gender', 'age', 'occupation', 'zip'])

/Users/andrewlarin/anaconda2/lib/python2.7/site-packages/ipykernel_launcher.py:1: ParserWarning: Falling back to the 'python' engine because the 'c' engine does not support regex separators (separators > 1 char and different from '\s+' are interpreted as regex); you can avoid this warning by specifying engine='python'.
  """Entry point for launching an IPython kernel.


In [3]:
users.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 6040 entries, 0 to 6039
Data columns (total 5 columns):
user          6040 non-null int64
gender        6040 non-null object
age           6040 non-null int64
occupation    6040 non-null int64
zip           6040 non-null object
dtypes: int64(3), object(2)
memory usage: 236.0+ KB


In [4]:
users.head(5)

,user,gender,age,occupation,zip
0,1,F,1,10,48067
1,2,M,56,16,70072
2,3,M,25,15,55117
3,4,M,45,7,02460
4,5,M,25,20,55455


In [5]:
ratings = pd.read_csv('ml-1m/ratings.dat', sep='::', names=['user', 'movie', 'rating', 'time'])

/Users/andrewlarin/anaconda2/lib/python2.7/site-packages/ipykernel_launcher.py:1: ParserWarning: Falling back to the 'python' engine because the 'c' engine does not support regex separators (separators > 1 char and different from '\s+' are interpreted as regex); you can avoid this warning by specifying engine='python'.
  """Entry point for launching an IPython kernel.


In [6]:
ratings.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1000209 entries, 0 to 1000208
Data columns (total 4 columns):
user      1000209 non-null int64
movie     1000209 non-null int64
rating    1000209 non-null int64
time      1000209 non-null int64
dtypes: int64(4)
memory usage: 30.5 MB


In [7]:
ratings.head(5)

,user,movie,rating,time
0,1,1193,5,978300760
1,1,661,3,978302109
2,1,914,3,978301968
3,1,3408,4,978300275
4,1,2355,5,978824291


In [8]:
movies = pd.read_csv('ml-1m/movies.dat', sep='::', names=['movie', 'title', 'genre'])

/Users/andrewlarin/anaconda2/lib/python2.7/site-packages/ipykernel_launcher.py:1: ParserWarning: Falling back to the 'python' engine because the 'c' engine does not support regex separators (separators > 1 char and different from '\s+' are interpreted as regex); you can avoid this warning by specifying engine='python'.
  """Entry point for launching an IPython kernel.


In [9]:
movies.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 3883 entries, 0 to 3882
Data columns (total 3 columns):
movie    3883 non-null int64
title    3883 non-null object
genre    3883 non-null object
dtypes: int64(1), object(2)
memory usage: 91.1+ KB


In [10]:
movies.head(5)

,movie,title,genre
0,1,Toy Story (1995),Animation|Children's|Comedy
1,2,Jumanji (1995),Adventure|Children's|Fantasy
2,3,Grumpier Old Men (1995),Comedy|Romance
3,4,Waiting to Exhale (1995),Comedy|Drama
4,5,Father of the Bride Part II (1995),Comedy


#### Как видно, жанры представленны в виде строки, что не очень удобно для поиска. Преобразуем это свойство в другой тип данных - множество:

In [11]:
movies['genre'] = movies['genre'].map(lambda x: set([x.strip() for x in x.split('|')]))

In [12]:
movies.head(5)

,movie,title,genre
0,1,Toy Story (1995),"{Comedy, Children's, Animation}"
1,2,Jumanji (1995),"{Children's, Adventure, Fantasy}"
2,3,Grumpier Old Men (1995),"{Romance, Comedy}"
3,4,Waiting to Exhale (1995),"{Drama, Comedy}"
4,5,Father of the Bride Part II (1995),{Comedy}


### 2. Оценить распределения (предлагается построить распределения количества оценок по фильмам и пользователям и увидеть т.н. "длинные хвосты" в распределениях):

#### Когда примерно стало понятно с какими данными предстоит работать, соединим их в один датасет, чтобы можно было провести более подробное исследование:

In [13]:
movie_lens = pd.merge(pd.merge(ratings, users), movies)

In [14]:
movie_lens.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 1000209 entries, 0 to 1000208
Data columns (total 10 columns):
user          1000209 non-null int64
movie         1000209 non-null int64
rating        1000209 non-null int64
time          1000209 non-null int64
gender        1000209 non-null object
age           1000209 non-null int64
occupation    1000209 non-null int64
zip           1000209 non-null object
title         1000209 non-null object
genre         1000209 non-null object
dtypes: int64(6), object(4)
memory usage: 83.9+ MB


In [15]:
movie_lens.head(5)

,user,movie,rating,time,gender,age,occupation,zip,title,genre
0,1,1193,5,978300760,F,1,10,48067,One Flew Over the Cuckoo's Nest (1975),{Drama}
1,2,1193,5,978298413,M,56,16,70072,One Flew Over the Cuckoo's Nest (1975),{Drama}
2,12,1193,4,978220179,M,25,12,32793,One Flew Over the Cuckoo's Nest (1975),{Drama}
3,15,1193,4,978199279,M,25,7,22903,One Flew Over the Cuckoo's Nest (1975),{Drama}
4,17,1193,5,978158471,M,50,1,95350,One Flew Over the Cuckoo's Nest (1975),{Drama}


In [16]:
ratings_total = movie_lens.groupby(['movie', 'title'])['movie'] \
    .agg(['count']) \
    .reset_index()
ratings_total.head(10)

,movie,title,count
0,1,Toy Story (1995),2077
1,2,Jumanji (1995),701
2,3,Grumpier Old Men (1995),478
3,4,Waiting to Exhale (1995),170
4,5,Father of the Bride Part II (1995),296
5,6,Heat (1995),940
6,7,Sabrina (1995),458
7,8,Tom and Huck (1995),68
8,9,Sudden Death (1995),102
9,10,GoldenEye (1995),888


#### Посмотрим распределение:

In [17]:
ratings_total['rounded_count'] = ratings_total['count'].map(lambda x: (x // 100) * 100)

In [18]:
ratings_total.head(10)

,movie,title,count,rounded_count
0,1,Toy Story (1995),2077,2000
1,2,Jumanji (1995),701,700
2,3,Grumpier Old Men (1995),478,400
3,4,Waiting to Exhale (1995),170,100
4,5,Father of the Bride Part II (1995),296,200
5,6,Heat (1995),940,900
6,7,Sabrina (1995),458,400
7,8,Tom and Huck (1995),68,0
8,9,Sudden Death (1995),102,100
9,10,GoldenEye (1995),888,800


In [19]:
ratings_total['rounded_count'].value_counts(normalize=True)

0       0.455208
100     0.160011
200     0.099298
300     0.070426
400     0.048300
500     0.032110
600     0.027253
700     0.022126
800     0.016190
900     0.013222
1000    0.011333
1100    0.009174
1300    0.007016
1200    0.005666
1400    0.004587
1700    0.003238
1500    0.002698
1600    0.002428
2200    0.001889
2500    0.001619
1900    0.000809
2000    0.000809
2100    0.000809
2300    0.000809
2600    0.000809
2900    0.000540
1800    0.000540
2400    0.000540
2800    0.000270
3400    0.000270
Name: rounded_count, dtype: float64

Как видно, почти половина фильмов имеет меньше 100 голосов зрителей. 16% - от 100 до 200 голосов, в остальном, распределение очень неравномерно.

### 3. Посмотреть на смещение оценок, увидеть сколько пользователей занижают оценки и завышают оценки (посмотреть разницу со средним значением, например):

In [20]:
vote_average = movie_lens['rating'].mean()
vote_average

3.581564453029317

In [21]:
def compare_mean_rating(x):
    diff = x - vote_average
    if diff > 0.5:
        return 1
    elif diff < -0.5:
        return -1
    else:
        return 0

In [22]:
users_average = movie_lens.groupby(['user'])['rating'] \
    .agg(['mean']) \
    .reset_index()
users_average['compare_mean_rating'] = users_average['mean'].map(compare_mean_rating)
users_average.head(5)

,user,mean,compare_mean_rating
0,1,4.188679,1
1,2,3.713178,0
2,3,3.901961,0
3,4,4.190476,1
4,5,3.146465,0


In [23]:
users_average['compare_mean_rating'].value_counts(normalize=True)

 0    0.732616
 1    0.186258
-1    0.081126
Name: compare_mean_rating, dtype: float64

Как видно, большинство пользователей ставит оценку в пределах +- 0.5 от средней оценки всего датасета, еще 18 процентов склонны завышать оценку и только 8 процентов - занижать.

### 4. Оценить средние оценки по фильмам и по пользователям:

#### Не очень понятно что здесь имеется ввиду, поэтому объединю его со следующим пунктом: 

### 5. Выбрать критерий, показывающий, что фильм нравится всем, или не нравится никому, и вывести топ фильмов из этих списков. Знаете ли вы эти фильмы, согласны ли с оценками?

#### Чтобы учесть успешность/неуспешность фильма надо учитывать 2 параметра:
#### 1) Средний рейтинг фильма;
#### 2) Количество проголосовавших за этот фильм;
#### Учитывать пользователей занижающих или завышающих рейтинг мы не будем, т.к. из количество не очень велико. Но при этом будем учитывать минимальное количество голосов, для которых можно считать оценку, так как очень много фильмов с малым количеством голосов и они могут сильно исказить истинную картину.
#### Сама же формула выглядит следующим образом:

<p>взвешенный вес (WR) = (v / (v+m)) * R + (m / (v+m)) * C</p>
<p></p>
<p>где:</p>
<ul>
<li>R = Средний рейтинг фильма</li>
<li>v = Количество проголосовавших за этот фильм</li>
<li>m = Минимальное количество голосов для оценка</li>
<li>C = Средний рейтинг для всей выборки</li>
</ul>

In [24]:
del users_average

In [25]:
ratings_total = ratings_total.rename(columns={'count': 'votes_count'})[['movie', 'votes_count']]

In [26]:
ratings_average = movie_lens.groupby(['movie'])['rating'] \
    .agg(['mean']) \
    .reset_index()

In [27]:
ratings_average = ratings_average.rename(columns={'mean': 'rating_average'})[['movie', 'rating_average']]

In [28]:
movie_ratings = pd.merge(pd.merge(movies, ratings_total), ratings_average)

In [29]:
movie_ratings.head(5)

,movie,title,genre,votes_count,rating_average
0,1,Toy Story (1995),"{Comedy, Children's, Animation}",2077,4.146846
1,2,Jumanji (1995),"{Children's, Adventure, Fantasy}",701,3.201141
2,3,Grumpier Old Men (1995),"{Romance, Comedy}",478,3.016736
3,4,Waiting to Exhale (1995),"{Drama, Comedy}",170,2.729412
4,5,Father of the Bride Part II (1995),{Comedy},296,3.006757


#### Средний рейтинг по выборке мы уже посчитали выше:

In [30]:
C = vote_average

#### Минимальным количеством голосов будет 0,75-квантиль, т.е. фильм должен иметь больше голосов чем 75% фильмов в выборке:

In [31]:
m = ratings_total['votes_count'].quantile(0.75)
m

350.0

In [32]:
def weighted_rating(x):
    v = x['votes_count']
    R = x['rating_average']
    return (v / (v + m) * R) + (m / (m + v) * C)

In [33]:
del ratings_average
del movie_lens

In [34]:
movie_ratings['wr'] = movie_ratings.apply(weighted_rating, axis=1)

In [35]:
movie_ratings.head(5)

,movie,title,genre,votes_count,rating_average,wr
0,1,Toy Story (1995),"{Comedy, Children's, Animation}",2077,4.146846,4.065327
1,2,Jumanji (1995),"{Children's, Adventure, Fantasy}",701,3.201141,3.327828
2,3,Grumpier Old Men (1995),"{Romance, Comedy}",478,3.016736,3.255492
3,4,Waiting to Exhale (1995),"{Drama, Comedy}",170,2.729412,3.302976
4,5,Father of the Bride Part II (1995),{Comedy},296,3.006757,3.318185


#### Выведем 15 топ фильмов из списка:

In [36]:
movie_ratings.sort_values('wr', ascending=False)[['title', 'genre', 'votes_count', 'rating_average', 'wr']] \
    .head(15)

,title,genre,votes_count,rating_average,wr
309,"Shawshank Redemption, The (1994)",{Drama},2227,4.554558,4.422409
802,"Godfather, The (1972)","{Action, Drama, Crime}",2223,4.524966,4.396637
513,Schindler's List (1993),"{Drama, War}",2304,4.510417,4.387923
1108,Raiders of the Lost Ark (1981),"{Action, Adventure}",2514,4.477725,4.368208
49,"Usual Suspects, The (1995)","{Thriller, Crime}",1783,4.517106,4.363595
253,Star Wars: Episode IV - A New Hope (1977),"{Action, Fantasy, Adventure, Sci-Fi}",2991,4.453694,4.362331
2557,"Sixth Sense, The (1999)",{Thriller},2459,4.406263,4.303506
713,Dr. Strangelove or: How I Learned to Stop Worr...,"{Sci-Fi, War}",1367,4.449890,4.272887
851,Casablanca (1942),"{Drama, Romance, War}",1669,4.412822,4.268721
579,"Silence of the Lambs, The (1991)","{Drama, Thriller}",2578,4.351823,4.259750


В общем и целом, с рейтингом я согласен, хотя не все фильмы мне знакомы, но поискав некоторые из них в гугле, оказалось, что рейтинги у них довольно высокие.

### 6. Выбрать лучшие фильмы:

#### Для того, чтобы выбрать лучше фильмы, повысим минимальное количеством голосов до 0,99-квантиль, т.е. оставим 1% самых популярных фильмов:

In [37]:
m = ratings_total['votes_count'].quantile(0.99)
m

1784.8999999999996

In [38]:
movie_ratings['wr_99'] = movie_ratings.apply(weighted_rating, axis=1)

In [39]:
movie_ratings.sort_values('wr_99', ascending=False)[['title', 'genre', 'votes_count', 'rating_average', 'wr_99']] \
    .head(15)

,title,genre,votes_count,rating_average,wr_99
253,Star Wars: Episode IV - A New Hope (1977),"{Action, Fantasy, Adventure, Sci-Fi}",2991,4.453694,4.127753
309,"Shawshank Redemption, The (1994)",{Drama},2227,4.554558,4.121672
1108,Raiders of the Lost Ark (1981),"{Action, Adventure}",2514,4.477725,4.105640
513,Schindler's List (1993),"{Drama, War}",2304,4.510417,4.104951
802,"Godfather, The (1972)","{Action, Drama, Crime}",2223,4.524966,4.104827
2651,American Beauty (1999),"{Drama, Comedy}",3428,4.317386,4.065440
2557,"Sixth Sense, The (1999)",{Thriller},2459,4.406263,4.059411
49,"Usual Suspects, The (1995)","{Thriller, Crime}",1783,4.517106,4.049086
579,"Silence of the Lambs, The (1991)","{Drama, Thriller}",2578,4.351823,4.036704
1848,Saving Private Ryan (1998),"{Action, Drama, War}",2653,4.337354,4.033379


Как мне кажется, рейтинг получился вполне содержательный, хотя "Звездные войны" на первом месте выглядят немного странно.